# Question 1:
### Import the necessary libraries


Solution:

In [3]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

# Question 2:
### Download the Dataset
Solution:
Dataset Downloaded and uploaded to drive https://www.kaggle.com/code/kredy10/simple-lstm-for-textclassification/data

# Question 3:
### Read dataset and do pre-processing
Solution:
Read dataset


In [4]:
df=pd.read_csv('C:\SANMIKA\IBM Project\spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


## Pre-processing the Dataset

In [6]:
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [7]:
X=df.v2
Y=df.v1
le=LabelEncoder()
Y=le.fit_transform(Y)
Y=Y.reshape(-1,1)

In [8]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.15)

In [10]:
max_words=1000
max_len=150
tok=Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences=tok.texts_to_sequences(X_train)
sequences_matrix=pad_sequences(sequences,maxlen=max_len)

In [11]:
inputs=Input(name='inputs',shape=[max_len])
layer=Embedding(max_words,50,input_length=max_len)(inputs)
layer=LSTM(64)(layer)
layer=Dense(256,name='FC1')(layer)
layer=Activation('relu')(layer)
layer=Dropout(0.5)(layer)
layer=Dense(1,name='out_layer')(layer)
layer=Activation('sigmoid')(layer)
model=Model(inputs=inputs,outputs=layer)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [12]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

## Train and Fit the Model

In [13]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,validation_split=0.2)

Epoch 1/10
30/30 [==============================] - 17s 319ms/step - loss: 0.3365 - accuracy: 0.8770 - val_loss: 0.1369 - val_accuracy: 0.9473
Epoch 2/10
30/30 [==============================] - 8s 270ms/step - loss: 0.0932 - accuracy: 0.9770 - val_loss: 0.0442 - val_accuracy: 0.9863
Epoch 3/10
30/30 [==============================] - 8s 269ms/step - loss: 0.0526 - accuracy: 0.9855 - val_loss: 0.0377 - val_accuracy: 0.9873
Epoch 4/10
30/30 [==============================] - 8s 278ms/step - loss: 0.0403 - accuracy: 0.9881 - val_loss: 0.0401 - val_accuracy: 0.9873
Epoch 5/10
30/30 [==============================] - 9s 283ms/step - loss: 0.0324 - accuracy: 0.9894 - val_loss: 0.0375 - val_accuracy: 0.9842
Epoch 6/10
30/30 [==============================] - 8s 268ms/step - loss: 0.0232 - accuracy: 0.9929 - val_loss: 0.0339 - val_accuracy: 0.9863
Epoch 7/10
30/30 [==============================] - 8s 261ms/step - loss: 0.0185 - accuracy: 0.9942 - val_loss: 0.0374 - val_accuracy: 0.9863
Epoch

## Save the Model

In [16]:
model.save('sms_classifier.h5')

## Preprocessing the Test Dataset

In [17]:
test_sequences=tok.texts_to_sequences(X_test)
test_sequences_matrix=pad_sequences(test_sequences,maxlen=max_len)

## Testing the Model

In [18]:
accr=model.evaluate(test_sequences_matrix,Y_test)

27/27 [==============================] - 1s 41ms/step - loss: 0.0382 - accuracy: 0.9928


In [20]:
print('Test set')
print('LOSS : {:0.3f} \n ACCURACY : {:0.3f}'.format(accr[0],accr[1])) 


Test set
LOSS : 0.038 
 ACCURACY : 0.993
